# **7. Perceptron**
--------
인공신경망(Neuron)의 한 종류.

## **Neuron?**
동물 신경계, 특히 인간의 뇌의 뉴런을 본 따 만든 망을 뉴런(인공 신경망)이라고 함.   

**뉴런의 동작 원리**   
1. **입력 신호의 총량이 threshold를 넘으면** 뉴런이 활성화 됨.
2. 활성화 된 뉴런은 신호를 다른 뉴런으로 전달.
3. 만약 입력 신호 총량이 threshold를 넘지 못하면, 신호가 다음 뉴런으로 전달 되지 않음.

## **Perceptron**
Perceptron의 초창기 모델은 linear classifier로, AND, OR 문제를 해결하기 위해 만들어 졌다.   
(2가지 class가 있을 때 이를 linear하게 분리하는)   

1. 입력이 들어왔을 때
2. 입력에 가중치들을 곱하게 되고
3. 이 입력 값들이 가중치를 넘으면
4. 활성화 함수(sigmoid 같은)를 거쳐서
5. 최종적인 output이 나온다.

## **Perceptron과 XOR**
하하 AND, OR문제? 문제 없다! 무적이다! 아무나 와라!   
XOR: 히히 과연 그럴까?   

Winsky: 너네 1개 layer Perceptron으로는 XOR 문제 절대 못 푼다. 여러 layer를 쌓아야 풀 수 있음.   
근데 각 층의 weight들 학습해야 하는데 너네 어떻게 할래?   
절.대.못.할.걸   
<br>

1개 층의 Perceptron으로는 못 푼다는걸 직접 느껴봐요

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

X = torch.FloatTensor([[0, 0],[0, 1], [1, 0],[1, 1]])
Y = torch.FloatTensor([[0],[1],[1],[0]])

linear = torch.nn.Linear(2, 1, bias = True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear, sigmoid).to(device)

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

for step in range(10001):
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if step % 100 == 0:
        print(step, cost.item()) #특정 지점부터 cost값이 줄지 않기 시작

0 0.6999189853668213
100 0.6931477785110474
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996
700 0.6931471824645996
800 0.6931471824645996
900 0.6931471824645996
1000 0.6931471824645996
1100 0.6931471824645996
1200 0.6931471824645996
1300 0.6931471824645996
1400 0.6931471824645996
1500 0.6931471824645996
1600 0.6931471824645996
1700 0.6931471824645996
1800 0.6931471824645996
1900 0.6931471824645996
2000 0.6931471824645996
2100 0.6931471824645996
2200 0.6931471824645996
2300 0.6931471824645996
2400 0.6931471824645996
2500 0.6931471824645996
2600 0.6931471824645996
2700 0.6931471824645996
2800 0.6931471824645996
2900 0.6931471824645996
3000 0.6931471824645996
3100 0.6931471824645996
3200 0.6931471824645996
3300 0.6931471824645996
3400 0.6931471824645996
3500 0.6931471824645996
3600 0.6931471824645996
3700 0.6931471824645996
3800 0.6931471824645996
3900 0.6931471824645996
4000 0.6931471824645996
4100 0.6931471824645996
4200

In [4]:
with torch.no_grad():
    hypothesis = model(X)
    prediction = hypothesis.float() >= torch.FloatTensor([0.5])
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ',
          predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[0.5]
 [0.5]
 [0.5]
 [0.5]] 
Correct:  [[0.]
 [0.]
 [0.]
 [0.]] 
Accuracy:  0.5


NN에서 Forward Propagation으로 linear가 2개일 때 W1, W2, b1, b2를 구한다고 하면,
$$linear1: K(X) = sigmoid(XW₁+B₁)$$
$$linear2: H(X) = sigmoid(K(X)W₂+b₂))$$
이런 식이 나올텐데, 이걸 어떻게 구하지..?

## **Backpropagation**
걱정 마세요 여러분 제가 해결해 드립니다(Chain rule)   
Weight에 대한 Loss의 미분 값을 구하고, Loss값을 최소화 시킬 수 있도록 조정함.    
편미분을 통해서 어떤 어려운 식이라도 간단하게 미분 가능!    
<br><br>

층을 더 쌓아봅시다!

In [4]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

X = torch.FloatTensor([[0, 0],[0, 1], [1, 0],[1, 1]])
Y = torch.FloatTensor([[0],[1],[1],[0]])

#nn.Linear 2층을 사용한다
w1 = torch.Tensor(2, 2).to(device)
b1 = torch.Tensor(2).to(device)
w2 = torch.Tensor(2, 1).to(device)
b2 = torch.Tensor(1).to(device)

torch.nn.init.normal_(w1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(w2)
torch.nn.init.normal_(b2)

learning_rate =1

def sigmoid(x):
    return 1.0 / (1.0 + torch.exp(-x))

def sigmoid_prime(x): # 시그모이드 미분한 메서드
    return sigmoid(x) * (1 - sigmoid(x))

In [5]:
for step in range(10001):
    l1 = torch.add(torch.matmul(X, w1), b1)
    a1 = sigmoid(l1)
    l2 = torch.add(torch.matmul(a1, w2), b2)
    Y_pred = sigmoid(l2)

    cost = -torch.mean(Y * torch.log(Y_pred) + (1 - Y) * torch.log(1 - Y_pred)) # BCE Loss

    #Back prop (chain rule)
    d_Y_pred = (Y_pred - Y) / (Y_pred * (1.0 - Y_pred) + 1e-7)
    d_l2 = d_Y_pred *sigmoid_prime(l2)
    d_b2 = d_l2
    d_w2 = torch.matmul(torch.transpose(a1, 0, 1), d_b2)

    d_a1 = torch.matmul(d_b2, torch.transpose(w2, 0, 1))
    d_l1 = d_a1 * sigmoid_prime(l1)
    d_b1 = d_l1
    d_w1 = torch.matmul(torch.transpose(X, 0, 1),d_b1)

    w1 = w1 - learning_rate*d_w1
    b1 = b1 - learning_rate*torch.mean(d_b1,0)
    w2 = w2 - learning_rate*d_w2
    b2 = b2 - learning_rate*torch.mean(d_b2,0)

    if step % 100 == 0:
        print(step, cost.item())

0 0.6940568089485168
100 0.281848669052124
200 0.0478261262178421
300 0.021345559507608414
400 0.01334218867123127
500 0.009612534195184708
600 0.007480262778699398
700 0.006107822991907597
800 0.00515343900769949
900 0.00445280596613884
1000 0.0039172363467514515
1100 0.003494983073323965
1200 0.003153689205646515
1300 0.0028723436407744884
1400 0.002636490622535348
1500 0.002435920527204871
1600 0.002263369970023632
1700 0.0021133646368980408
1800 0.001981841865926981
1900 0.0018655199091881514
2000 0.001762002008035779
2100 0.0016691621858626604
2200 0.0015856071840971708
2300 0.0015098856529220939
2400 0.0014410398434847593
2500 0.0013781121233478189
2600 0.0013204144779592752
2700 0.0012673037126660347
2800 0.0012183014769107103
2900 0.0011728692334145308
3000 0.0011306634405627847
3100 0.0010913999285548925
3200 0.0010547498241066933
3300 0.0010204592254012823
3400 0.0009882738813757896
3500 0.0009580745827406645
3600 0.0009296368807554245
3700 0.0009027967462316155
3800 0.000877

In [7]:
print((Y_pred> 0.5).float())

tensor([[0.],
        [1.],
        [1.],
        [0.]])


### Code: xor-nn

In [8]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

X = torch.FloatTensor([[0, 0],[0, 1], [1, 0],[1, 1]])
Y = torch.FloatTensor([[0],[1],[1],[0]])

linear1 = torch.nn.Linear(2, 2, bias = True) # 두 개를 쌓았다
linear2 = torch.nn.Linear(2, 1, bias = True)
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid).to(device)

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

for step in range(10001):
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if step % 1000 == 0:
        print(step, cost.item())

0 0.7016617059707642
1000 0.4316394329071045
2000 0.018728366121649742
3000 0.009086793288588524
4000 0.005975094623863697
5000 0.004445049446076155
6000 0.003536670934408903
7000 0.002935562049970031
8000 0.0025085590314120054
9000 0.0021896709222346544
10000 0.001942584989592433


In [9]:
with torch.no_grad():
    hypothesis = model(X)
    prediction = hypothesis.float() >= torch.FloatTensor([0.5])
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ',
          predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[0.00169352]
 [0.99773765]
 [0.99772567]
 [0.00153141]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0
